In [1]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

np_ar = np.asarray(df)
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [ ]:
print(df.columns)
print(df.Temperature.describe())

In [ ]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

# print(df)
variety_dummies = pd.get_dummies(df.Variety)
# print(dummies)
df = pd.concat([df, variety_dummies], axis=1)


In [2]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_date = plant_date.rename("Season")
plant_date = pd.to_datetime(plant_date)
plant_date = plant_date.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, plant_date], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])


In [ ]:
# df['Plant date'] = df['Plant date'].apply(lambda dt: (dt.month%12 + 3)//3)

In [3]:
df['Plant date']

0        2
1        2
2        2
3        2
4        2
5        2
6        2
7        2
8        2
9        2
10       2
11       2
12       2
13       2
14       2
15       2
16       2
17       2
18       2
19       2
20       2
21       2
22       2
23       2
24       2
25       2
26       2
27       2
28       2
29       2
        ..
82006    3
82007    3
82008    3
82009    3
82010    3
82011    3
82012    3
82013    3
82014    3
82015    3
82016    3
82017    3
82018    3
82019    3
82020    3
82021    3
82022    3
82023    3
82024    3
82025    3
82026    3
82027    3
82028    3
82029    3
82030    3
82031    3
82032    3
82033    3
82034    3
82035    3
Name: Plant date, Length: 80086, dtype: int64

In [4]:
print(df.columns)
print(df.isnull().sum())

Index(['Experiment', 'Location', 'Variety', 'Planting date', 'Yield',
       'Check Yield', 'Yield difference', 'Year', 'Latitude', 'Longitude',
       'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class', 'CEC',
       'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'PI', 'Area',
       'Plant date'],
      dtype='object')
Experiment          0
Location            0
Variety             0
Planting date       0
Yield               0
Check Yield         0
Yield difference    0
Year                0
Latitude            0
Longitude           0
Temperature         0
Precipitation       0
Solar Radiation     0
Soil class          0
CEC                 0
Organic matter      0
pH                  0
Clay                0
Silt                0
Sand                0
PI                  0
Area                0
Plant date          0
dtype: int64


In [5]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_).rename(index=int, columns={0: "Loc Clust 0", 1: "Loc Clust 1", 2: "Loc Clust 2", 3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [6]:
print(df.isnull().sum())

Experiment          1950
Location            1950
Variety             1950
Planting date       1950
Yield               1950
Check Yield         1950
Yield difference    1950
Year                1950
Latitude            1950
Longitude           1950
Temperature         1950
Precipitation       1950
Solar Radiation     1950
Soil class          1950
CEC                 1950
Organic matter      1950
pH                  1950
Clay                1950
Silt                1950
Sand                1950
PI                  1950
Area                1950
Plant date          1950
Loc Clust 0         1950
Loc Clust 1         1950
Loc Clust 2         1950
Loc Clust 3         1950
dtype: int64


In [7]:
print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]

Index(['Experiment', 'Location', 'Variety', 'Planting date', 'Yield',
       'Check Yield', 'Yield difference', 'Year', 'Latitude', 'Longitude',
       'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class', 'CEC',
       'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'PI', 'Area',
       'Plant date', 'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2',
       'Loc Clust 3'],
      dtype='object')


In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
binarized = lb.fit(df.Variety)
print(binarized)
df.Variety = pd.Series(binarized.transform(df.Variety))
print(binarized.transform(df.Variety).reshape(1,-1))

In [8]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'Variety', 'PI', 'Planting date']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep = ['Silt', 'Precipitation', 'Temperature']
MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, columns_to_keep + MUST_HAVE_COLUMNS] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print(df.columns)

Index(['Yield', 'Year', 'Temperature', 'Precipitation', 'Solar Radiation',
       'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay', 'Silt', 'Sand',
       'Area', 'Plant date', 'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2',
       'Loc Clust 3', 'YieldBucket'],
      dtype='object')


In [10]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

We expect to be 0 nan values and there actually are 0 nan values

Yield              0
Year               0
Temperature        0
Precipitation      0
Solar Radiation    0
Soil class         0
CEC                0
Organic matter     0
pH                 0
Clay               0
Silt               0
Sand               0
Area               0
Plant date         0
Loc Clust 0        0
Loc Clust 1        0
Loc Clust 2        0
Loc Clust 3        0
YieldBucket        0
dtype: int64
Yield <class 'numpy.float64'>
Year <class 'numpy.float64'>
Temperature <class 'numpy.float64'>
Precipitation <class 'numpy.float64'>
Solar Radiation <class 'numpy.float64'>
Soil class <class 'numpy.float64'>
CEC <class 'numpy.float64'>
Organic matter <class 'numpy.float64'>
pH <class 'numpy.float64'>
Clay <class 'numpy.float64'>
Silt <class 'numpy.float64'>
Sand <class 'numpy.float64'>
Area <class 'numpy.float64'>
Plant date <class 'numpy.float64'>
Loc Clust 0 <class 'numpy.float64'>
Loc Clust 1 <class 'numpy.float64

In [11]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt', 'Sand', 'Area',
       'Plant date', 'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2',
       'Loc Clust 3'],
      dtype='object')


In [12]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

X_train shape: (7813, 17) 
y_train shape: (7813,)


In [13]:
# This function will evaluate the errors based on RMSE (from the challenge spec)
# also will evaluate based on average error

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100)
#     print("Average Error: ", np.mean(avg_error_vector))
    print("Average Error details:\n", avg_error_vector.describe())

In [15]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

evaluate_errors(preds, y_test)

RMSE Error:  7.569704667297518
Average Error details:
 count    3907.000000
mean       10.705794
std        11.583171
min         0.007068
25%         3.436881
50%         7.834823
75%        13.862608
max       219.736525
Name: Yield, dtype: float64


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [19]:
pd.get_dummies(df['Plant date']).sum()

2.0    62372
3.0    15742
4.0       22
dtype: int64

In [16]:
# GET OUTPUT OF FEATURE IMPORTANCE

feature_importances = regr.feature_importances_
feature_importances = pd.Series(feature_importances)
feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
for index, row in feature_importance_df.iterrows():
    print(row['feature'], 'has importance: ', row['feature_importance'])


Soil class has importance:  0.01733528048975531
Loc Clust 0 has importance:  0.017618173355426158
Loc Clust 3 has importance:  0.018926084966503366
Loc Clust 1 has importance:  0.01957171783899267
Loc Clust 2 has importance:  0.019668117194292456
Plant date has importance:  0.03201553224375252
Clay has importance:  0.034371709380952915
Sand has importance:  0.04476461719153968
CEC has importance:  0.057496437342252384
pH has importance:  0.05931139328608352
Area has importance:  0.06479587829164976
Solar Radiation has importance:  0.08298906786646205
Year has importance:  0.08359305647515082
Temperature has importance:  0.08913024637760716
Organic matter has importance:  0.09146143108245394
Precipitation has importance:  0.10483892087297993
Silt has importance:  0.1621123357441453


In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]




# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')


In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = mean_squared_error(preds, y_test)
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
    print(errors)
#     print(errors.describe())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))